objectives
- find the top hit for each contig in the mmseqs output, across all dbs
- determine what percentage of contigs were classified
- determine what percentage of reads were classified

find the top hit for each contig in the blast output

In [ ]:
import Pkg
pkgs = [
    "Revise",
    "DataFrames",
    "StatsBase",
    "ProgressMeter",
    "uCSV"
]
# Pkg.activate(; temp=true)
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [ ]:
base_dir = dirname(pwd())
data_dir = joinpath(base_dir, "data")
sample_directories = readdir(joinpath(data_dir, "SRA"), join=true)

In [ ]:
easy_taxonomy_lca_reports = String[]
for sample_directory in sample_directories
    mmseqs_directory_contents = readdir(joinpath(sample_directory, "mmseqs_easy_taxonomy"), join=true)
    sample_lca_reports = filter(x -> occursin("final.contigs.fastg.gfa.fna.mmseqs_easy_taxonomy.", x) && occursin("_lca.tsv", x), mmseqs_directory_contents)
    append!(easy_taxonomy_lca_reports, sample_lca_reports)
end
easy_taxonomy_lca_reports

In [ ]:
lca_tsv = first(easy_taxonomy_lca_reports)

In [ ]:
# replace(replace(basename(lca_tsv), "final.contigs.fastg.gfa.fna." => ""), "_lca.tsv" => "")

In [ ]:
viral_tax_ids = Set(parse.(Int, filter(!isempty, readlines(`conda run -n taxonkit taxonkit list --ids 10239 --indent ""`))))

In [ ]:
joint_lca_table = DataFrames.DataFrame()
ProgressMeter.@showprogress for lca_tsv in easy_taxonomy_lca_reports
    method = replace(replace(basename(lca_tsv), "final.contigs.fastg.gfa.fna." => ""), "_lca.tsv" => "")
    # @show method
    this_lca_table = Mycelia.parse_mmseqs_easy_taxonomy_lca_tsv(lca_tsv)
    this_lca_table[!, "sample"] .= basename(dirname(dirname(lca_tsv)))
    this_lca_table[!, "method"] .= method
    append!(joint_lca_table, this_lca_table)
end
sort!(joint_lca_table, "contig_id")

In [ ]:
# StatsBase.describe(joint_lca_table[!, "support -log(E-value)"])

In [ ]:
# -log(0.3)

In [ ]:
joint_lca_table = joint_lca_table[map(x -> x in viral_tax_ids, joint_lca_table[!, "taxon_id"]), :]

In [ ]:
joint_lca_table[!, "taxon_id"]

In [ ]:
# NCBI host metadata
ncbi_metadata_file = joinpath(dirname(pwd()), "metadata", "NCBI-virus-refseq.transformed.tsv")
ncbi_host_metadata = DataFrames.DataFrame(uCSV.read(ncbi_metadata_file, header=1, delim='\t'))

In [ ]:
joint_lca_ncbi_table = DataFrames.innerjoin(joint_lca_table, ncbi_host_metadata, on="taxon_id" => "taxid")
joint_lca_ncbi_table[1:1, :]

In [ ]:
count(parse.(Bool, joint_lca_ncbi_table[!, "host_is_vertebrate"]))

In [ ]:
count(parse.(Bool, joint_lca_ncbi_table[!, "host_is_mammal"]))

In [ ]:
count(parse.(Bool, joint_lca_ncbi_table[!, "host_is_primate"]))

In [ ]:
count(parse.(Bool, joint_lca_ncbi_table[!, "host_is_human"]))

In [ ]:
joint_lca_ncbi_table[!, "host_is_human"] = parse.(Bool, joint_lca_ncbi_table[!, "host_is_human"])
joint_lca_ncbi_table = joint_lca_ncbi_table[joint_lca_ncbi_table[!, "host_is_human"], :]

In [ ]:
for x in sort(unique(joint_lca_ncbi_table[!, "taxon_name"]))
    println(x)
end

In [ ]:
# ICTV host metadata
ictv_metadata_file = joinpath(dirname(pwd()), "metadata", "VMR_MSL38_v1 - VMR MSL38 v1.transformed.tsv")
ictv_host_metadata = DataFrames.DataFrame(uCSV.read(ictv_metadata_file, header=1, delim='\t', typedetectrows=100))
ictv_host_metadata = ictv_host_metadata[.!isempty.(ictv_host_metadata[!, "taxid"]), :]
ictv_host_metadata[!, "taxid"] = parse.(Int, ictv_host_metadata[!, "taxid"])
joint_lca_ictv_table = DataFrames.innerjoin(joint_lca_table, ictv_host_metadata, on="taxon_id" => "taxid")

# StatsBase.countmap(joint_lca_ictv_table[!, "Host source"])

In [ ]:
joint_lca_ictv_table_hits = joint_lca_ictv_table[joint_lca_ictv_table[!, "Host source"] .== "vertebrates", :]
for x in sort(unique(joint_lca_ictv_table_hits[!, "Virus name(s)"]))
    println(x)
end

In [ ]:
# # VirusHostDB metadata
virushostdb_metadata_file = joinpath(dirname(pwd()), "metadata", "virushostdb.transformed.tsv")
virushostdb_metadata = DataFrames.DataFrame(uCSV.read(virushostdb_metadata_file, header=1, delim='\t', typedetectrows=1086, encodings=Dict("missing" => missing)))

In [ ]:
joint_lca_virushostdb_table = DataFrames.innerjoin(joint_lca_table, virushostdb_metadata, on="taxon_id" => "virus_taxid")

In [ ]:
count(parse.(Bool, joint_lca_virushostdb_table[!, "host_is_vertebrate"]))

In [ ]:
count(parse.(Bool, joint_lca_virushostdb_table[!, "host_is_mammal"]))

In [ ]:
count(parse.(Bool, joint_lca_virushostdb_table[!, "host_is_primate"]))

In [ ]:
count(parse.(Bool, joint_lca_virushostdb_table[!, "host_is_human"]))

In [ ]:
joint_lca_virushostdb_table[!, "host_is_human"] = parse.(Bool, joint_lca_virushostdb_table[!, "host_is_human"])
joint_lca_virushostdb_table = joint_lca_virushostdb_table[joint_lca_virushostdb_table[!, "host_is_human"], :]

In [ ]:
sort(unique(joint_lca_virushostdb_table[!, "taxon_name"]))

In [ ]:
joint_lca_ictv_table_hits[!, "Virus name(s)"]

In [ ]:
StatsBase.countmap(vcat(
    joint_lca_virushostdb_table[!, "taxon_name"],
    joint_lca_ncbi_table[!, "taxon_name"],
    ))

In [ ]:
for x in unique(sort(union(
    sort(unique(joint_lca_virushostdb_table[!, "taxon_name"])),
    sort(unique(joint_lca_ncbi_table[!, "taxon_name"]))
)))
    println(x)
end

In [ ]:
for x in sort(unique(joint_lca_ictv_table_hits[!, "Virus name(s)"]))
    println(x)
end